# Machine Learning in Neuronal Modeling: A Comparative Analysis and Implementation

Understanding how neurons respond to stimuli is a fundamental problem in neuroscience. This project explores whether we can predict spike counts (number of action potentials) in human neurons using machine learning, based on stimulus waveforms and voltage responses. 
The motivation behind this work is twofold:
- To apply computational models in neurophysiology and understand stimulus-spike relationships
- To evaluate the viability of artificial neural networks (ANNs) in modeling biological neural activit

### Biological and Computational Motivation

Spike trains are key to how neurons encode and transmit information. However, even when the same stimulus is presented to different neurons, the spiking behavior can vary dramatically.

By training models to predict spike counts, we aim to identify patterns in stimulus-response relationships, with the long-term goal of contributing toward better brain-computer interfaces and neuroprosthetics.

Artificial neural networks (ANNs) seemed like a natural starting point because they are biologically inspired and flexible enough to model complex non-linear systems.

### Dataset Overview: Allen Cell Types Database (Human Neurons)

The Allen Brain Atlas is a large-scale open-access repository of standardized electrophysiology data. For this project, we used the human dataset from the Cell Types Database. 
**Key dataset details:**
- Source: [Allen Cell Types Database](https://celltypes.brain-map.org/)
- Species: Human (Homo sapiens)
- Number of neurons: 413
- Data includes: Stimulus waveform (pA), voltage trace (mV), spike timings, metadata (cell ID, sweep ID)

The electrophysiology sweeps consist of current-clamp recordings in response to step or ramp current injections.

## Why Machine Learning?

Traditional approaches for modeling neuronal spiking involve biophysical simulations like the Hodgkin-Huxley model, which require detailed ion channel kinetics and are computationally intensive. In contrast, machine learning can abstract away from those low-level details and attempt to capture spiking patterns directly from observed data.

This is particularly useful when working with large datasets like AllenDB, and allows us to test the feasibility of predictive modeling without explicit mechanistic descriptions.
